In [ ]:
# upgrade stuffs
# tensorflow upgrade could be slow if running in notebook. try in BASH and see if it's faster.

# upgrade tensorflow before running this notebook
# pip install -q --upgrade tensorflow
# in emulator the above command can be prefixed with !
# this should upgrade tensorflow to 1.7.0

!pip install -q --upgrade tensorflow

In [1]:
from __future__ import absolute_import, division, print_function

import os
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe

print("TensorFlow version: {}".format(tf.VERSION))
print("numpy version: {}", np.__version__)
print("Eager execution: {}".format(tf.executing_eagerly()))

Instructions for updating:
Use the retry module or similar alternatives.
TensorFlow version: 1.7.0
numpy version: {} 1.14.2
Eager execution: False


In [ ]:
# turn on eager execution in this box
# do this only when necessary
tf.enable_eager_execution()
print("Eager execution: {}".format(tf.executing_eagerly()))

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)
sess = tf.Session()
# the next line could cause errer if eager execution is enabled.
if not tf.executing_eagerly(): print(sess.run([node1, node2]))
# node3 = tf.add(node1, node2)
node3 = node1 + node2
print("node3:", node3)
if not tf.executing_eagerly(): print("sess.run(node3):", sess.run(node3))

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)
[3.0, 4.0]
node3: Tensor("add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


In [ ]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

In [ ]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

In [ ]:
train_dataset_url = "http://download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Local copy of the dataset file: {}".format(train_dataset_fp))

In [ ]:
!head -n5 {train_dataset_fp}

In [ ]:
!ls -al

In [3]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

# event file name is like events.out.tfevents.{timestamp}.{hostname}

In [ ]:
# must run this in BASH
# otherwise cannot cancel

!tensorboard --logdir .